In [40]:
import json

with open("sample_config.json", "r") as sample_file:
    template = json.load(sample_file)
    
class RefPath:
    def __init__(self, ref_path):
        paths = ref_path.split("#/info/x-pods/")
        parts = paths[1].split("/")
        self.pod_name = parts[0]
        self.container_name = parts[2]

    @property
    def full_path(self):
        return "#/info/x-pods/" + self.pod_name + "/containers/" + self.container_name + "/port"


class Method:
    def __init__(self, path_name, method_name, ref_path, schema_name, full_method):
        self.path_name = path_name
        self.method_name = method_name
        self.ref_path = ref_path        
        self.schema_name = schema_name
        self.full_method = full_method        

    def __str__(self):
        return self.path_name + ": " + self.method_name + " (" + self.ref_path.full_path + ")" +" <-> " + self.schema_name
    
class Pod:
    def __init__(self, pod_name, full_pod):
        self.pod_name = pod_name
        self.full_pod = full_pod

In [15]:
methods = []
for path_name, path in template["paths"].items():
    for method_name, method in path.items():        
        ref_path = RefPath(method["x-location"]["$ref"])
        all_references = list(set(gen_dict_extract('$ref', method)))
        schema_name = get_schema_only(all_references)    
        
        methods.append(Method(path_name, method_name, ref_path, schema_name, method))

pods = []
for pod_name, pod in template["info"]["x-pods"].items():
    full_pod = dict()
    full_pod[pod_name] = pod
    pods.append(Pod(pod_name, full_pod))

In [ ]:
for()

In [8]:
import copy
templates = []
for pod in pods:
    sample_template = copy.deepcopy(template)
    pod_methods = {}
    for method in methods:
        if method.ref_path.pod_name == pod.pod_name:
            pod_methods.setdefault(method.path_name, {})
            pod_methods[method.path_name][method.method_name] = method.full_method
    if len(pod_methods):
        sample_template["info"]["x-pods"] = pod.full_pod
        sample_template["paths"] = pod_methods

        templates.append(sample_template)

In [22]:
def gen_dict_extract(key, var):
    if hasattr(var,'items'):
        for k, v in var.items():
            if k == key:
                yield v
            if isinstance(v, dict):                
                for result in gen_dict_extract(key, v):
                    yield result
            elif isinstance(v, (list)):
                for d in v:
                    for result in gen_dict_extract(key, d):
                        yield result                        


def get_schema_only(references):
    saved_schema = 'default'
    for reference in references:
        schema = reference.split("#/components/schemas/")        
        try:
            saved_schema = schema[1]
        except IndexError:
            pass
    
    return saved_schema

In [14]:
schema_methods = {}
for method in methods:
    all_references = list(set(gen_dict_extract('$ref', method.full_method)))
    
    schema_name = get_schema_only(all_references)    
    
    schema_methods.setdefault(schema_name, [])
    schema_methods[schema_name].append(method)

schema_methods


{'Pet': [<__main__.Method at 0x7fbd884cc730>,
 'default': [<__main__.Method at 0x7fbd884cc880>,
 'ApiResponse': [<__main__.Method at 0x7fbd884cc640>],
 'Order': [<__main__.Method at 0x7fbd884cca00>,
 'User': [<__main__.Method at 0x7fbd884ccfd0>,
  <__main__.Method at 0x7fbd883ec310>]}

In [23]:
get_schema_only(['#/info/x-pods/pod1/containers/c1/port', '#/components/schemas/Pet'])


'Pet'

In [25]:
# creating initial configuration start from here
import json

with open("01config.json", "r") as initial_config:
    initial_template = json.load(initial_config)
    

In [43]:
# getting all methods from a fat container

methods = []
for path_name, path in initial_template["paths"].items():
    for method_name, method in path.items():        
        ref_path = RefPath(method["x-location"]["$ref"]) 
        
        all_references = list(set(gen_dict_extract('$ref', method)))        
        schema_name = get_schema_only(all_references)            
        
        methods.append(Method(path_name, method_name, ref_path, schema_name, method))

pods = []
for pod_name, pod in initial_template["info"]["x-pods"].items():
    full_pod = dict()
    full_pod[pod_name] = pod
    pods.append(Pod(pod_name, full_pod))

In [50]:
pods[0].full_pod[pod_name]["containers"]

{'c1': {'id': 'c1', 'metrics': {'load': ''}, 'port': 4000}}

In [55]:
schema_methods = {}
for method in methods:  
    schema_methods.setdefault(method.schema_name, [])
    schema_methods[method.schema_name].append(method)    

c1
c1
c1
c1
c1
c1


In [70]:
import copy

copied_template = copy.deepcopy(initial_template)

index = 1
for schema_name, methods in schema_methods.items():
    if index == 1:
        pass        
    else:
        first_method = methods[0]
        new_container = "c" + str(index)
        
        container_template = {'id': new_container, 'metrics': {'load': ''} }
        
        copied_template["info"]["x-pods"][method.ref_path.pod_name]["containers"][new_container] = container_template
        
        for method in methods:
            method.ref_path.container_name = new_container
            method.full_method["x-location"]["$ref"] = method.ref_path.full_path
            copied_template["paths"][method.path_name][method.method_name] = method.full_method
    index +=1
    
copied_template

{'openapi': '3.0.0',
 'info': {'description': 'This is a sample server Petstore server.  You can find out more about     Swagger at [http://swagger.io](http://swagger.io) or on [irc.freenode.net, #swagger](http://swagger.io/irc/).      For this sample, you can use the api key `special-key` to test the authorization     filters.',
  'version': '1.0.0',
  'title': 'Swagger Petstore',
  'termsOfService': 'http://swagger.io/terms/',
  'contact': {'email': 'apiteam@swagger.io'},
  'license': {'name': 'Apache 2.0',
   'url': 'http://www.apache.org/licenses/LICENSE-2.0.html'},
  'x-pods': {'pod1': {'name': 'pod1',
    'metrics': {'CPU': '', 'RAM': ''},
    'containers': {'c1': {'id': 'c1', 'metrics': {'load': ''}, 'port': 4000},
     'c2': {'id': 'c2', 'metrics': {'load': ''}},
     'c3': {'id': 'c3', 'metrics': {'load': ''}},
     'c4': {'id': 'c4', 'metrics': {'load': ''}}}}}},
 'servers': [{'url': 'https://petstore.swagger.io',
   'description': 'Optional server description, e.g. Main (pro